In [1]:
# background: to extract info from XML files for RAPID LEAN CRF project.
# data needs to extract are:
# oversight_info/has_dmc,primary_outcome/measure, primary_outcome/time_frame, secondary_outcome/measure, secondary_outcome/time_frame,arm_group/arm_group_label[0],arm_group/description, eligibility/criteria/textblock
# put each extraction from the XML into one row in the csv file.

In [1]:
import csv
import xml.etree.ElementTree as ET
import re

In [2]:
# read the NCTID as in a list from the 69xlsx file
import pandas as pd
NCTID_list = []
df = pd.read_excel('input\CT69.xlsx', sheet_name='Sheet1')
NCTID_list = df['NCT Number'].dropna().tolist()
print (len(NCTID_list))

69


#### below codes ran for two folders : clinical trial and katsanos folder
input for clinical trial list:  input\\ClinicalTrial || output filename is CT_parse_69.csv

In [4]:
# get the elements from the XML file into a dictionary
import xml.etree.ElementTree as ET
#check if a file exist -- when parse xml error, the xml file will be moved to another folder
import os.path
from os import path

# function to get the primary outcome and secondary outcome
def CT_tag(element):
    element_count = len(root.findall(element))
    xml_dict[element+'_count'] = element_count

    # primary/secondary outcome info
    Out_list=[]
    Out_measure=""
    Out_time=""
    Out_desc=""
    for Out in root.iter(element):
        if Out.find("measure") is not None: 
            Out_measure = Out[0].text 
        if Out.find("time_frame") is not None: 
            Out_time = Out[1].text  
        if Out.find("descrption") is not None: 
            Out_desc = Out[2].text 
        Out_list.append(Out_measure+"|"+Out_time+"|"+Out_desc)
    xml_dict[element+'_list'] = Out_list

#function to get the arm_group label and description
def CT_arm(element):
    element_count = len(root.findall(element))
    xml_dict[element+'_count'] = element_count
    
    arm_list=[]
    arms_label=""
    arms_description=""
    for arms in root.iter(element):
        if arms.find("arm_group_label") is not None: 
            arms_label = arms[0].text 
        if arms.find("description") is not None:
            arms_description = arms[2].text
            arms_description=arms_description.replace("\n","")
        arm_list.append(arms_label+"|"+arms_description)
    xml_dict[element+"_list"]=arm_list

filepath = input('filepath for list of XML files: use \"folderundercurrentdirectory\\subfolder\":')
outfilename = input('please give the filename for output: ')

outputfile = "output\\"+outfilename

for i in NCTID_list:
    xml_dict={}
    filename = filepath+"\\"+i+".xml"
    if path.exists(filename):
        print ("processing "+filename)
        xml_dict['NCTID']=i
        with open(filename, mode='rt', errors='ignore') as xml:
            tree = ET.parse(xml)
            root = tree.getroot()
        #oversight_info
            if root.findall('oversight_info/has_dmc'): 
                for dmc in root.findall('oversight_info/has_dmc'):
                    if dmc != None:
                        xml_dict['dmc_oversight']=dmc.text
                    else:
                        xml_dict['dmc_oversight']="n/a"
            else:
                xml_dict['dmc_oversight']="n/a"
           
        #primary and secondary outcome
            CT_tag('primary_outcome')
            CT_tag('secondary_outcome')
            
        #arm info
            CT_arm('arm_group')

        #eligibility
            for inclusion_exclusion in root.iter('eligibility'):
                for text in inclusion_exclusion.findall('criteria/textblock') :
                    content = text.text
                    content=str(content)
                    content=content.replace("\n","")
                    content = re.sub(' +', ' ', content)
                    xml_dict['inclusion_exclusion']=content
           #mesh_term
            mesh_term_list = []
            if root.find('condition_browse') is not None:
                for mesh_term in root.findall('condition_browse/mesh_term'):
                    mesh_term_list.append(mesh_term.text)
            if root.find('intervention_browse') is not None:
                for mesh_term in root.findall('intervention_browse/mesh_term'):
                    mesh_term_list.append(mesh_term.text)
            xml_dict['mesh_term'] = mesh_term_list   
            
        print (i+" parsed successfully!")
        #print (xml_dict)
     
    # write the dictionary into a csv file, each key:value is a cell, separated by ";". one NCTID in one row
        with open(outputfile, 'a') as f:
            #writer = csv.writer(f)  
            for key, value in xml_dict.items():
                if type(xml_dict[key]) == list:
                    f.write(str(key)+":")
                    for t in xml_dict[key]:
                        f.write(str(t)+",")
                    f.write("||")
                else:
                    f.write(str(key)+":"+str(value))
                f.write("||")
            f.write("\n")
    f.close()    

filepath for list of XML files: use "folderundercurrentdirectory\subfolder":input\\ClinicalTrial
please give the filename for output: CT_69_parse_1227.csv
processing input\\ClinicalTrial\NCT02599389.xml
NCT02599389 parsed successfully!
processing input\\ClinicalTrial\NCT02710656.xml
NCT02710656 parsed successfully!
processing input\\ClinicalTrial\NCT01221610.xml
NCT01221610 parsed successfully!
processing input\\ClinicalTrial\NCT01867736.xml
NCT01867736 parsed successfully!
processing input\\ClinicalTrial\NCT00696956.xml
NCT00696956 parsed successfully!
processing input\\ClinicalTrial\NCT00930813.xml
NCT00930813 parsed successfully!
processing input\\ClinicalTrial\NCT01594684.xml
NCT01594684 parsed successfully!
processing input\\ClinicalTrial\NCT02498080.xml
NCT02498080 parsed successfully!
processing input\\ClinicalTrial\NCT01558505.xml
NCT01558505 parsed successfully!
processing input\\ClinicalTrial\NCT03175744.xml
NCT03175744 parsed successfully!
processing input\\ClinicalTrial\NCT

#### Katsanos list now:
input for Katsanos list: input\\katsanos || output file name is katsanos.csv
note: the katsanos.csv contains longer XML file list than what in the katsanos folder. If not find in the katsanos folder, should copy from the output in clinicaltrial result (as the overlaps)

In [5]:
#read the Katsanos list
NCTID_list = []
df = pd.read_excel('input\katsanos.xlsx', sheet_name='Sheet1')
NCTID_list = df['NCT Number'].dropna().tolist()
print (len(NCTID_list))

20


In [6]:
#this is a repeated cell, just wanted to keep the results for Katsanos 
# get the elements from the XML file into a dictionary
import xml.etree.ElementTree as ET
#check if a file exist -- when parse xml error, the xml file will be moved to another folder
import os.path
from os import path

# function to get the primary outcome and secondary outcome
def CT_tag(element):
    element_count = len(root.findall(element))
    xml_dict[element+'_count'] = element_count

    # primary/secondary outcome info
    Out_list=[]
    Out_measure=""
    Out_time=""
    Out_desc=""
    for Out in root.iter(element):
        if Out.find("measure") is not None: 
            Out_measure = Out[0].text 
        if Out.find("time_frame") is not None: 
            Out_time = Out[1].text  
        if Out.find("descrption") is not None: 
            Out_desc = Out[2].text 
        Out_list.append(Out_measure+"|"+Out_time+"|"+Out_desc)
    xml_dict[element+'_list'] = Out_list

#function to get the arm_group label and description
def CT_arm(element):
    element_count = len(root.findall(element))
    xml_dict[element+'_count'] = element_count
    
    arm_list=[]
    arms_label=""
    arms_description=""
    for arms in root.iter(element):
        if arms.find("arm_group_label") is not None: 
            arms_label = arms[0].text 
        if arms.find("description") is not None:
            arms_description = arms[2].text
            arms_description=arms_description.replace("\n","")
        arm_list.append(arms_label+"|"+arms_description)
    xml_dict[element+"_list"]=arm_list

filepath = input('filepath for list of XML files: use \"folderundercurrentdirectory\\subfolder\":')
outfilename = input('please give the filename for output: ')

outputfile = "output\\"+outfilename

for i in NCTID_list:
    xml_dict={}
    filename = filepath+"\\"+i+".xml"
    if path.exists(filename):
        print ("processing "+filename)
        xml_dict['NCTID']=i
        with open(filename, mode='rt', errors='ignore') as xml:
            tree = ET.parse(xml)
            root = tree.getroot()
        #oversight_info
            if root.findall('oversight_info/has_dmc'): 
                for dmc in root.findall('oversight_info/has_dmc'):
                    if dmc != None:
                        xml_dict['dmc_oversight']=dmc.text
                    else:
                        xml_dict['dmc_oversight']="n/a"
            else:
                xml_dict['dmc_oversight']="n/a"
           
        #primary and secondary outcome
            CT_tag('primary_outcome')
            CT_tag('secondary_outcome')
            
        #arm info
            CT_arm('arm_group')

        #eligibility
            for inclusion_exclusion in root.iter('eligibility'):
                for text in inclusion_exclusion.findall('criteria/textblock') :
                    content = text.text
                    content=str(content)
                    content=content.replace("\n","")
                    content = re.sub(' +', ' ', content)
                    xml_dict['inclusion_exclusion']=content
            
        #mesh_term
            mesh_term_list = []
            if root.find('condition_browse') is not None:
                for mesh_term in root.findall('condition_browse/mesh_term'):
                    mesh_term_list.append(mesh_term.text)
            if root.find('intervention_browse') is not None:
                for mesh_term in root.findall('intervention_browse/mesh_term'):
                    mesh_term_list.append(mesh_term.text)
            xml_dict['mesh_term'] = mesh_term_list
        # write the dictionary into a csv file, each key:value is a cell, separated by ";". one NCTID in one row
        print (i+" parsed successfully!")
        #print (xml_dict)
    
        with open(outputfile, 'a') as f:
            #writer = csv.writer(f)  
            for key, value in xml_dict.items():
                if type(xml_dict[key]) == list:
                    f.write(str(key)+":")
                    for t in xml_dict[key]:
                        f.write(str(t)+",")
                    f.write("||")
                else:
                    f.write(str(key)+":"+str(value))
                f.write("||")
            f.write("\n")
    f.close()    

filepath for list of XML files: use "folderundercurrentdirectory\subfolder":input\\katsanos
please give the filename for output: katsanos_parse_1227.csv
processing input\\katsanos\NCT01816412.xml
NCT01816412 parsed successfully!
processing input\\katsanos\NCT02540018.xml
NCT02540018 parsed successfully!
processing input\\katsanos\NCT01083030.xml
NCT01083030 parsed successfully!
processing input\\katsanos\NCT01850056.xml
NCT01850056 parsed successfully!
processing input\\katsanos\NCT01450722.xml
NCT01450722 parsed successfully!
processing input\\katsanos\NCT00156624.xml
NCT00156624 parsed successfully!
processing input\\katsanos\NCT00120406.xml
NCT00120406 parsed successfully!
processing input\\katsanos\NCT03023098.xml
NCT03023098 parsed successfully!


In [24]:
# add from the CT_parse_69.csv to katsanos_parse.csv what is not in the input folder
CT_list =[]
for i in NCTID_list:
    if path.exists("input\\katsanos\\"+i+".xml"):
        print (i+".xml file is already parsed." )
    else:
        CT_list.append(i)
print (len(CT_list))

NCT01816412.xml file is already parsed.
NCT02540018.xml file is already parsed.
NCT01083030.xml file is already parsed.
NCT01850056.xml file is already parsed.
NCT01450722.xml file is already parsed.
NCT00156624.xml file is already parsed.
NCT00120406.xml file is already parsed.
NCT03023098.xml file is already parsed.
12
